# Raster Download and Terrain Analysis with WCS

This notebook demonstrates a workflow for working with raster data obtained through a Web Coverage Service (WCS). Specifically, it involves the following steps:

1. **Connecting and downloading raster data**: Accessing a WCS to download a raster dataset corresponding to a specific geographic extent defined by a GeoJSON file.
2. **Generating raster derivatives**: Creating various derived products from the downloaded raster, including:
    * Slope map
    * Aspect map
    * Hillshade map

These derived products are useful for topographic analysis, terrain modeling, and geospatial studies in applications such as environmental, geological, and urban research. The notebook is designed to automate this workflow and streamline the analysis process.

## 1. Read in the geojson file and WCS connection

First of all, we'll import all the libraries needed and read the geojson which stores the extension of our study area.

In [1]:
# Import libraries
from owslib.wcs import WebCoverageService
from osgeo import gdal
import os
import parameters
import geopandas as gpd
import glob

In [2]:
# Read the file as a geodataframe
polygon = gpd.read_file(parameters.input_file)

In [3]:
# Bounding boxes of the polygon
minx, miny, maxx, maxy = polygon.total_bounds

bbox = (minx, miny, maxx, maxy)

In [4]:
# WCS connection
wcs = WebCoverageService(parameters.wcs_url, version='1.0.0')

# Spatial Resolution
resolution_x = 5  # 5 meters
resolution_y = 5  # 5 meters

In [5]:
# Identifier an title for each layer in the service
for layer_id, layer_info in wcs.contents.items():
    print(f"Identifier: {layer_id}, Title: {layer_info.title}")

Identifier: 1, Title: MDS_LIDAR_1M_EGUNERATUENA_MAS_ACTUALIZADA
Identifier: 2, Title: MDT_LIDAR_1M_EGUNERATUENA_MAS_ACTUALIZADA
Identifier: 3, Title: MDT_LIDAR_5M_EGUNERATUENA_MAS_ACTUALIZADA
Identifier: 4, Title: MDT_LIDAR_25M_EGUNERATUENA_MAS_ACTUALIZADA
Identifier: 5, Title: ORIENTACIONES_LIDAR_25M_EGUNERATUENA_MAS_ACTUALIZADA
Identifier: 6, Title: PENDIENTES_LIDAR_25M_EGUNERATUENA_MAS_ACTUALIZADA
Identifier: 7, Title: MDS_LIDAR_2008_1M
Identifier: 8, Title: MDT_LIDAR_2008_1M
Identifier: 9, Title: MDT_LIDAR_2008_5M
Identifier: 10, Title: MDT_LIDAR_2008_25M
Identifier: 11, Title: MDS_LIDAR_2012_1M
Identifier: 12, Title: MDT_LIDAR_2012_1M
Identifier: 13, Title: MDT_LIDAR_2012_5M
Identifier: 14, Title: MDT_LIDAR_2012_25M
Identifier: 15, Title: MDT_LIDAR_2013_1M
Identifier: 16, Title: MDT_LIDAR_2013_5M
Identifier: 17, Title: MDT_LIDAR_2013_25M
Identifier: 18, Title: MDT_LIDAR_2016_1M
Identifier: 19, Title: MDT_LIDAR_2016_5M
Identifier: 20, Title: MDT_LIDAR_2016_25M
Identifier: 21, Title

We will select the third layer, which contains the digital elevation model (DEM) with a 5 meter resolution for the entire Basque Country. It is worth mentioning that this raster was obtained through an aerial survey equipped with LiDAR.

In [6]:
# Select the layer with the local coordinate system and the resolution mentioned above
response = wcs.getCoverage(
    identifier='3',
    bbox=bbox,
    crs='EPSG:25830',
    format='GeoTIFF',
    resx=resolution_x,
    resy=resolution_y,
    timeout=180
)

In [7]:
# Raster path
mdt_path = os.path.join(parameters.output_dir, 'clipped_mdt.tif')

# Create the folder 
if not os.path.exists(parameters.output_dir):
    os.makedirs(parameters.output_dir)

# Store as a GeoTIFF
with open(mdt_path, 'wb') as f:
    f.write(response.read())

## 2. Slope, Aspect and Hillsahed maps

Once we clipped and download the DEM, it's time to create the derivative maps using the  ```gdal.DEMProcessing``` command. It is worth noting that this function needs to be executed twice to work correctly.

In [8]:
# Define raster directories
slope_output_path = os.path.join(parameters.output_dir, 'slope', 'slope.tif')
aspect_output_path = os.path.join(parameters.output_dir, 'aspect', 'aspect.tif')
hillshade_output_path = os.path.join(parameters.output_dir, 'hillshade', 'hillshade.tif')

# Create folders
os.makedirs(os.path.dirname(slope_output_path), exist_ok=True)
os.makedirs(os.path.dirname(aspect_output_path), exist_ok=True)
os.makedirs(os.path.dirname(hillshade_output_path), exist_ok=True)

### 2.1. Slope

In [9]:
# Read DEM using gdal
mdt_name = os.path.join(parameters.output_dir, 'clipped_mdt.tif')
dem = gdal.Open(mdt_name)

c:\Users\User\miniconda3\envs\gis\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [10]:
# Slope
slope = gdal.DEMProcessing(slope_output_path, dem, 'slope', computeEdges=True, zFactor=1)

In [11]:
# Run the function twice
slope = gdal.DEMProcessing(slope_output_path, dem, 'slope', computeEdges=True, zFactor=1)

### 2.2. Aspect

In [12]:
# Aspect
aspect = gdal.DEMProcessing(aspect_output_path, dem, 'aspect', computeEdges=True, zFactor=1, scale=1)

In [13]:
# Run the function twice
aspect = gdal.DEMProcessing(aspect_output_path, dem, 'aspect', computeEdges=True, zFactor=1)

### 2.3. Hillshade

In [14]:
# Hillshade
hillshade = gdal.DEMProcessing(hillshade_output_path, dem, 'hillshade', computeEdges=True, zFactor=1, altitude=45, scale=1)

In [15]:
# Run the function twice
hillshade = gdal.DEMProcessing(hillshade_output_path, dem, 'hillshade', computeEdges=True, zFactor=1, altitude=45, scale=1)